# Часть 2: Создание собственного MCP-сервера

В этом примере мы создадим собственный MCP-сервер с использованием FastMCP и подключим его к агенту через STDIO транспорт.

MCP решает проблемы обычного tool calling через стандартизацию и переиспользуемость.

## Шаг 1: Импорты и настройка

In [1]:
import asyncio
import os
from pathlib import Path

from dotenv import load_dotenv
from fastmcp import Client
from langchain.agents import create_agent
from langchain_mcp_adapters.tools import load_mcp_tools
from langchain_openai import ChatOpenAI

load_dotenv()

BASE_MODEL = os.getenv("BASE_MODEL") or "qwen/qwen3-235b-a22b-2507"

<frozen abc>:106: LangGraphDeprecatedSinceV10: AgentStatePydantic has been moved to `langchain.agents`. Please update your import to `from langchain.agents import AgentStatePydantic`. Deprecated in LangGraph V1.0 to be removed in V2.0.


## Шаг 2: Структура MCP-сервера

Наш сервер находится в файле `server_stdio.py`. Он использует FastMCP и предоставляет два инструмента:

- `convert_document` - конвертирует документы в markdown
- `analyze_document` - возвращает статистику о документе

Сервер запускается как отдельный процесс и общается через STDIO транспорт.

## Шаг 3: Подключение к серверу

Создаем клиент, который подключается к нашему серверу через STDIO.
Сервер автоматически предоставляет список своих инструментов через `tools/list`.

In [2]:
async def main():
    server_script = Path("server_stdio.py")

    async with Client(str(server_script)) as client:
        print("Available tools:")
        tools_list = await client.list_tools()
        for tool in tools_list:
            print(f"  - {tool.name}")

        tools = await load_mcp_tools(client.session)

        llm = ChatOpenAI(model=BASE_MODEL, temperature=0)

        agent = create_agent(llm, tools)

        test_pdf = Path("../../data/test.pdf")

        response = await agent.ainvoke(
            {
                "messages": [
                    (
                        "user",
                        f"Analyze the document structure at {test_pdf} and create laconic summary of the text.",
                    )
                ]
            }
        )

        print("Agent response:")
        for message in response["messages"]:
            if (
                message.type == "ai"
                and hasattr(message, "content")
                and message.content
            ):
                print(f"\n{message.content}")

## Шаг 4: Запуск примера

Выполним асинхронную функцию main для взаимодействия с сервером.

In [4]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
asyncio.run(main())

Available tools:
  - convert_document
  - analyze_document
Agent response:

The manul (Otocolobus manul) is a small wild cat species adapted to cold, arid steppe and semi-desert ecosystems across Central and Western Asia. It exhibits unique morphological and behavioral traits, including dense fur for insulation, cryptic coloration for camouflage, and reliance on rocky outcrops and burrows for shelter. The species is solitary, with males having larger home ranges (avg. 98.8 km²) than females (avg. 23.1 km²) in Mongolia, and shows seasonal breeding patterns tied to photoperiod.

Manuls primarily feed on small mammals, especially pikas, which constitute over 50% of their diet. They employ various hunting techniques such as stalking, ambush, and flushing prey. Despite a broad geographic range, populations are sparse and patchily distributed due to habitat specialization, predation pressure, and low prey availability.

Threats include predation by raptors, foxes, wolves, and domestic dogs, 